<a href="https://colab.research.google.com/github/dookda/cmu_lab_154743/blob/main/python_workshop/python_15_raster_geospatial_on_cloud_ndvi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap
# conda install -c conda-forge earthengine-api
# conda install geemap

In [5]:
import ee
import geemap

# Authenticate the Earth Engine library.
try:
    ee.Initialize(project="ee-sakda-451407")  # uses cached credentials if available
except Exception:
    ee.Authenticate()
    ee.Initialize(project="ee-sakda-451407")


In [ ]:
Map = geemap.Map()
Map.set_center(99, 18.80, 8)
Map.add_basemap("HYBRID")
Map.addLayer(ee.FeatureCollection([]), {}, "Empty")
Map


In [ ]:
# 1) Load & filter
aoi = ee.Geometry.Polygon([[[98.8, 18.6], [99.2, 18.6], [99.2, 19.1], [98.8, 19.1], [98.8, 18.6]]])
start, end = '2024-01-01', '2024-03-31'
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(aoi) \
        .filterDate(start, end)

# 2) Cloud/cirrus mask using QA60
def mask_s2_sr(image):
    return (image.divide(10000)
                 .select(['B2','B3','B4','B8'], ['blue','green','red','nir'])
                 .copyProperties(image, image.propertyNames()))

s2_clean   = s2.map(mask_s2_sr)
median_rgb = s2_clean.median().clip(aoi)

# 3) NDVI
ndvi = median_rgb.normalizedDifference(['nir','red']).rename('NDVI')

# 4) Visualize
vis_rgb  = {'min':0.03, 'max':0.30, 'bands':['red','green','blue']}
vis_ndvi = {'min':0.0,  'max':0.8,  'palette':['#d73027','#fee08b','#1a9850']}

Map = geemap.Map(center=[18.9, 99.0], zoom=9)
Map.addLayer(median_rgb, vis_rgb,  'S2 RGB (Q1 2024)')
Map.addLayer(ndvi,      vis_ndvi, 'NDVI (Q1 2024)')
Map.addLayer(aoi, {'color':'yellow'}, 'AOI')
Map


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Build an ImageCollection with NDVI band
s2_ndvi = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(aoi).filterDate('2023-01-01', '2024-12-31') \
    .map(mask_s2_sr) \
    .map(lambda i: i.addBands(i.normalizedDifference(['nir','red']).rename('NDVI')))

# Reduce each image to mean NDVI over AOI -> Feature(date, NDVI)
def regional_mean(img):
    stats = img.select('NDVI').reduceRegion(
        reducer=ee.Reducer.mean(), geometry=aoi, scale=20, bestEffort=True)
    return ee.Feature(None, {
        'date': img.date().format('YYYY-MM-dd'),
        'NDVI': stats.get('NDVI')
    })

fc = ee.FeatureCollection(s2_ndvi.map(regional_mean))
# After (current API)
ndvi_df = geemap.ee_to_df(fc)
ndvi_df['date'] = pd.to_datetime(ndvi_df['date'])
ndvi_df['NDVI'] = pd.to_numeric(ndvi_df['NDVI'], errors='coerce')
ndvi_df = ndvi_df.dropna(subset=['NDVI']).sort_values('date')

# quick plot
import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))
plt.plot(ndvi_df['date'], ndvi_df['NDVI'])
plt.xlabel('Date')
plt.ylabel('Mean NDVI')
plt.title('S2 NDVI over AOI')
plt.grid(True)
plt.show()


In [ ]:
# GAUL Level-2 for Chiang Mai
adm2 = ee.FeatureCollection('FAO/GAUL/2015/level2') \
    .filter(ee.Filter.eq('ADM0_NAME','Thailand')) \
    .filter(ee.Filter.eq('ADM1_NAME','Chiang Mai'))

Map.addLayer(adm2, {}, 'Chiang Mai')

Map

In [23]:
# Mean NDVI per district from the Q1 composite
zonal = ndvi.reduceRegions(
    collection=adm2,
    reducer=ee.Reducer.mean(),
    scale=20,
    tileScale=2
)

zonal_df = geemap.ee_to_df(zonal)
zonal_df = zonal_df[['ADM2_NAME','mean']].rename(columns={'mean':'NDVI_mean'})
zonal_df.sort_values('NDVI_mean', ascending=False).head()


,ADM2_NAME,NDVI_mean
15,Phrao,0.629123
10,Mae Rim,0.609537
16,Samoeng,0.596579
11,Mae Taeng,0.595048
6,Hang Dong,0.565561


In [ ]:
# Convert property to an image
ndvi_img = zonal.reduceToImage(properties=['mean'], reducer=ee.Reducer.first())

vmin = float(zonal_df['NDVI_mean'].min())
vmax = float(zonal_df['NDVI_mean'].max())
pal  = ['#d73027','#fee08b','#1a9850']

Map = geemap.Map(center=[18.79, 98.99], zoom=8)
Map.addLayer(ndvi_img, {'min':vmin, 'max':vmax, 'palette':pal}, 'NDVI Mean by District')

Map.addLayer(adm2.style(**{'color':'333333','width':1,'fillColor':'00000000'}), {}, 'ADM2 boundaries')

legend_params = {"min": vmin, "max": vmax, "palette": pal}
Map.add_colorbar(legend_params, label="NDVI Mean (Q1 2024)")

Map


In [ ]:
# Export the NDVI composite
task_img = ee.batch.Export.image.toDrive(
    image=ndvi,
    description='chiangmai_ndvi_2024',
    folder='gee_exports',
    fileNamePrefix='ndvi_2024',
    region=aoi,
    scale=100,
    maxPixels=1e13
)
task_img.start()




In [ ]:
print("Image export started:", task_img.id)

# Export zonal statistics as CSV
task_tbl = ee.batch.Export.table.toDrive(
    collection=zonal,
    description='chiangmai_ndvi_zonal_2024',
    folder='gee_exports',
    fileNamePrefix='ndvi_zonal_2024',
    fileFormat='CSV'
)
task_tbl.start()
print("Table export started:", task_tbl.id)

In [ ]:
# (Optional) simple task monitor
import time
def wait_for_tasks(poll=15):
    while True:
        running = [t for t in ee.batch.Task.list() if t.status()['state'] in ('READY','RUNNING')]
        if not running:
            print("No running tasks.")
            break
        print(f"{len(running)} task(s) running...")
        time.sleep(poll)

wait_for_tasks()
